In [6]:
from httpx import ReadTimeout
import nba_api
import pandas as pd
import time
import pprint
import json
import os
from nba_api.stats.static import players
player_dict = players.get_players() #properties: id, full_name, first_name, last_name, is_active
#Only pull from players that played from 2000-2025
filtered_player_dict = []
filename = "all_nba_players.json"
try:
    with open(filename, 'w') as json_file:
        json.dump(player_dict, json_file, indent=4)
    print(f"Successfully wrote data to {filename}")
except IOError as e:
    print(f"Error writing to file {filename}: {e}")

Successfully wrote data to all_nba_players.json


In [7]:
def load_players_from_json(filename="all_nba_players.json"):
    with open(filename, 'r') as f:
        return json.load(f)

In [8]:
from nba_api.stats.endpoints import playercareerstats
from requests import ReadTimeout, ConnectionError
def fetch_career_stats():
    all_players = load_players_from_json()
    rows = []
    for p in all_players[:10]:
        retries = 3
        success = False
        while retries >0 and not success:
            try:
                career = playercareerstats.PlayerCareerStats(player_id=p["id"], per_mode36="PerGame", timeout = 30)
                print(f"Fetching data for {p['full_name']}")
                df = career.get_data_frames()[0]
                df["PLAYER_ID"], df["PLAYER_NAME"] = p["id"], p["full_name"]
                rows.append(df)
                print(f"Successfully added {p['full_name']}")
                success = True
                time.sleep(1.2)  # NBA API rate limiting
            except (ReadTimeout, ConnectionError) as e:
                retries -= 1
                print(f"{type(e).__name__} for {p['full_name']}, retries left: {retries-1}")
                time.sleep(6)
            except Exception as e:
                print(f"Hard Failure on {p['full_name']}: {e}")
                break
    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()

career_stats_df = fetch_career_stats()
career_stats_df.to_csv("nba_player_career_stats_sample.csv", index=False)

Fetching data for Alaa Abdelnaby
Successfully added Alaa Abdelnaby
Fetching data for Zaid Abdul-Aziz
Successfully added Zaid Abdul-Aziz
Fetching data for Kareem Abdul-Jabbar
Successfully added Kareem Abdul-Jabbar
Fetching data for Mahmoud Abdul-Rauf
Successfully added Mahmoud Abdul-Rauf
Fetching data for Tariq Abdul-Wahad
Successfully added Tariq Abdul-Wahad
Fetching data for Shareef Abdur-Rahim
Successfully added Shareef Abdur-Rahim
Fetching data for Tom Abernethy
Successfully added Tom Abernethy
Fetching data for Forest Able
Successfully added Forest Able
Fetching data for John Abramovic
Successfully added John Abramovic
Fetching data for Alex Abrines
Successfully added Alex Abrines


C:\Users\natha\AppData\Local\Temp\ipykernel_79780\4249043660.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()


In [8]:
import pandas as pd
df = pd.read_json("nba_player_stats_2000_2025.json")
modern_players = df["PLAYER_NAME"].unique().tolist()
all_players = load_players_from_json()
modern_player_dict = [p for p in all_players if p["full_name"] in modern_players]
with open("modern_nba_players.json", 'w') as f:
    json.dump(modern_player_dict, f, indent=4)


In [5]:
from nba_api.stats.endpoints import playerawards
def fetch_awards(player_id, player_name, retries=3):
    delay = 0.5
    for attempt in range(retries):
        try:
            obj = playerawards.PlayerAwards(player_id=player_id)
            df = obj.get_data_frames()[0]
            print(f"Fetched awards for {player_name}")
            return df
        except (ReadTimeout, ConnectionError) as e:
            time.sleep(delay)
            print(f"Error fetching awards for {player_name}: {e}")
            delay *= 2  # exponential backoff
        except Exception:
            return None
    return None
nba_player_awards = {}
for p in modern_player_dict:
    df = fetch_awards(p["id"], p["full_name"])
    if df is not None and not df.empty:
        nba_player_awards[p["full_name"]] = df.to_dict("records")
    time.sleep(0.25)  # tiny delay to stay safe

Fetched awards for Mahmoud Abdul-Rauf
Fetched awards for Tariq Abdul-Wahad
Fetched awards for Shareef Abdur-Rahim
Fetched awards for Alex Abrines
Fetched awards for Precious Achiuwa
Fetched awards for Alex Acker
Fetched awards for Quincy Acy
Fetched awards for Hassan Adams
Fetched awards for Jaylen Adams
Fetched awards for Jordan Adams
Fetched awards for Steven Adams
Fetched awards for Bam Adebayo
Fetched awards for Deng Adel
Fetched awards for Jeff Adrien
Fetched awards for Arron Afflalo
Fetched awards for Ochai Agbaji
Fetched awards for Maurice Ager
Fetched awards for Blake Ahearn
Fetched awards for Alexis Ajinca
Fetched awards for Josh Akognon
Fetched awards for DeVaughn Akoon-Purcell
Fetched awards for Solomon Alabi
Fetched awards for Santi Aldama
Fetched awards for Furkan Aldemir
Fetched awards for Cole Aldrich
Fetched awards for LaMarcus Aldridge
Fetched awards for Cliff Alexander
Fetched awards for Cory Alexander
Fetched awards for Courtney Alexander
Fetched awards for Joe Alexa

KeyboardInterrupt: 

need to fix this since and split up requests as to not get a timeout/connection error

In [2]:
!pip install aiohttp

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\natha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import asyncio
import time
from nba_api.stats.endpoints import playerawards
from requests.exceptions import ReadTimeout, ConnectionError

def fetch_awards_sync(player_id, player_name, timeout=5, retries=3):
    delay = 0.5
    for attempt in range(retries):
        try:
            obj = playerawards.PlayerAwards(player_id=player_id, timeout=timeout)
            df = obj.get_data_frames()[0]
            print(f"Fetched awards for {player_name}")
            return df
        except (ReadTimeout, ConnectionError) as e:
            print(f"Error fetching {player_name}: {e}")
            time.sleep(delay)
            delay *= 2
        except Exception as e:
            print(f"Unexpected error for {player_name}: {e}")
            return None
    return None

async def fetch_awards_async(player):
    # Run the blocking nba_api call in a thread
    df = await asyncio.to_thread(
        fetch_awards_sync,
        player["id"],
        player["full_name"]
    )
    if df is not None and not df.empty:
        return player["full_name"], df.to_dict("records")
    return player["full_name"], None

async def main(players, max_concurrency=10):
    sem = asyncio.Semaphore(max_concurrency)

    async def sem_task(p):
        async with sem:
            return await fetch_awards_async(p)

    tasks = [asyncio.create_task(sem_task(p)) for p in players]
    results = await asyncio.gather(*tasks)

    return {name: recs for name, recs in results if recs is not None}

# Run it
nba_player_awards = await main(modern_player_dict)

Error fetching Alex Abrines: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=5)
Error fetching Jaylen Adams: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=5)
Error fetching Precious Achiuwa: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=5)
Error fetching Quincy Acy: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=5)
Error fetching Tariq Abdul-Wahad: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=5)
Error fetching Alex Acker: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=5)
Error fetching Hassan Adams: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=5)
Error fetching Shareef Abdur-Rahim: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=5)
Error fetching Jordan Adams: HTTPSConnectionPool(host='stats.nba.com

In [8]:
traded_df = career_stats_df.copy()

marked down if a player was traded during a season or not

In [32]:
sample_df = pd.read_csv("nba_player_career_stats_sample.csv")
target_column = "TEAM_ABBREVIATION"

has_value_down_1 = sample_df[target_column].shift(-1) == "TOT"
has_value_down_2 = sample_df[target_column].shift(-2) == "TOT"

sample_df["traded_mid_season"] = has_value_down_1 | has_value_down_2
sample_df["traded_mid_season"].value_counts()
sample_df.to_csv("nba_player_career_stats_sample.csv", index=False)

In [15]:
all_df = pd.read_csv("nba_player_stats_2000_2025.csv")
all_df.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
       'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK',
       'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK',
       'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK',
       'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK',
       'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK',
       'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK',
       'DD2_RANK', 'TD3_RANK', 'WNBA_FANTASY_PTS_RANK', 'TEAM_COUNT',
       'SEASON'],
      dtype='object')